In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load dataset
digits = load_digits()
X = digits.images / 16.0
y = digits.target.reshape(-1, 1)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Reshape to 8x8x1 for CNN
X = X.reshape(-1, 8, 8, 1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Tiny CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), activation="relu", input_shape=(8,8,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(16, (3,3), activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                    validation_split=0.1, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

model.summary()

Epoch 1/30
41/41 [==============================] - 9s 30ms/step - loss: 2.2607 - accuracy: 0.1616 - val_loss: 2.2188 - val_accuracy: 0.2014
Epoch 2/30
41/41 [==============================] - 0s 5ms/step - loss: 2.1069 - accuracy: 0.3372 - val_loss: 2.0069 - val_accuracy: 0.3333
Epoch 3/30
41/41 [==============================] - 0s 4ms/step - loss: 1.7577 - accuracy: 0.5097 - val_loss: 1.5758 - val_accuracy: 0.5486
Epoch 4/30
41/41 [==============================] - 0s 4ms/step - loss: 1.2597 - accuracy: 0.7378 - val_loss: 1.0407 - val_accuracy: 0.7639
Epoch 5/30
41/41 [==============================] - 0s 4ms/step - loss: 0.8577 - accuracy: 0.8221 - val_loss: 0.7112 - val_accuracy: 0.8333
Epoch 6/30
41/41 [==============================] - 0s 5ms/step - loss: 0.6296 - accuracy: 0.8592 - val_loss: 0.5536 - val_accuracy: 0.8819
Epoch 7/30
41/41 [==============================] - 0s 4ms/step - loss: 0.4969 - accuracy: 0.8817 - val_loss: 0.4232 - val_accuracy: 0.8819
Epoch 8/30
41/41 [=

In [2]:
# Convert float32 TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("digits_cnn.tflite", "wb") as f:
    f.write(tflite_model)

# Quantize to int8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
with open("digits_cnn_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmp2wq548ln\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmp2wq548ln\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpv6r4vdty\assets


INFO:tensorflow:Assets written to: C:\Users\kukil\AppData\Local\Temp\tmpv6r4vdty\assets


In [3]:
!xxd -i digits_cnn_quant.tflite > digits_cnn_quant.h